In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In [2]:
df1 = pd.read_csv("Datasets/instagram_reach.csv")
df1.sample()

,Unnamed: 0,S.No,USERNAME,Caption,Followers,Hashtags,Time since posted,Likes
24,24,28,data_enthusiast,"Interesting, most of them prefer that because ...",166,#data_enthusiast #dataanalytics#datascientist ...,2 hours,10


In [3]:
df1.reset_index(drop=True)
df1["Time"] = df1["Time since posted"].apply(lambda x: int(x.split(" ")[0]))
df1.drop(columns=["Unnamed: 0","S.No", "Caption", "USERNAME","Time since posted"], inplace=True)

In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Followers  100 non-null    int64 
 1   Hashtags   100 non-null    object
 2   Likes      100 non-null    int64 
 3   Time       100 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 3.2+ KB


In [5]:
df1.describe()

,Followers,Likes,Time
count,100.00000,100.00000,100.000000
mean,961.96000,46.48000,3.460000
std,1014.62567,55.08698,3.394648
min,11.00000,8.00000,2.000000
25%,252.75000,19.00000,2.000000
50%,612.00000,29.00000,2.000000
75%,1197.00000,46.00000,3.000000
max,4496.00000,349.00000,24.000000


In [6]:
print(df1.loc[:0]["Hashtags"])

0    #MachineLearning #AI #DataAnalytics #DataScien...
Name: Hashtags, dtype: object


In [7]:
def hastag(text):
    ps = PorterStemmer()
    wordnet = WordNetLemmatizer()
    
    tags = nltk.sent_tokenize(text)
    corpus = []
    tag = re.sub("[^a-zA-Z]", " ", tags[0])
    tag = tag.lower()
    tag = tag.split()
#     tag = [wordnet.lemmatize(word) for word in tag if not word in set(stopwords.words("english"))]
    print(tag)
    tfid = TfidfVectorizer()
    
    X = tfid.fit_transform(tag).toarray()
    return X

In [8]:
hastag(df1.loc[0]["Hashtags"])

['machinelearning', 'ai', 'dataanalytics', 'datascienc', 'datalake']


array([[0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])

In [9]:
from gensim.models import Word2Vec

# Define the list of hashtags
hashtags = ['machinelearning', 'ai', 'dataanalytics', 'datascience', 'datalake']

# Create a list of lists containing the tokens
documents = [hashtag.split() for hashtag in hashtags]

# Train the Word2Vec model
model = Word2Vec(documents,  window=5, min_count=1, workers=4)

# # Get the word embeddings for each hashtag
embeddings = {hashtag: model.wv[hashtag] for hashtag in hashtags}
embeddings
# # Print the embeddings
# for hashtag, embedding in embeddings.items():
#     print(hashtag, embedding)


{'machinelearning': array([-0.00713902,  0.00124103, -0.00717672, -0.00224462,  0.0037193 ,
         0.00583312,  0.00119818,  0.00210273, -0.00411039,  0.00722533,
        -0.00630704,  0.00464722, -0.00821997,  0.00203647, -0.00497705,
        -0.00424769, -0.00310898,  0.00565521,  0.0057984 , -0.00497465,
         0.00077333, -0.00849578,  0.00780981,  0.00925729, -0.00274233,
         0.00080022,  0.00074665,  0.00547788, -0.00860608,  0.00058446,
         0.00686942,  0.00223159,  0.00112468, -0.00932216,  0.00848237,
        -0.00626413, -0.00299237,  0.00349379, -0.00077263,  0.00141129,
         0.00178199, -0.0068289 , -0.00972481,  0.00904058,  0.00619805,
        -0.00691293,  0.00340348,  0.00020606,  0.00475375, -0.00711994,
         0.00402695,  0.00434743,  0.00995737, -0.00447374, -0.00138926,
        -0.00731732, -0.00969783, -0.00908026, -0.00102275, -0.00650329,
         0.00484973, -0.00616403,  0.00251919,  0.00073944, -0.00339215,
        -0.00097922,  0.00997913